# Doppler imaging: Multi-component maps

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In this tutorial, we'll discuss how to model complex spatial-spectral stellar surfaces by instantiating a multi-component map. This allows us to model a star whose spectrum (not just intensity!) varies with position over the surface in (arbitrarily) complex ways.

## Creating a spectral-spatial map

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry

We'll start with a simple example: a star with a single dark spot, whose spectrum is different than the rest of the photosphere. Let's generate a map of the spot on a 2-dimensional latitude-longitude grid.

In [ ]:
lat = np.linspace(-90, 90, 300)
lon = np.linspace(-180, 180, 600)
image = np.ones((len(lat), len(lon)))
y = lat.reshape(-1, 1)
x = lon.reshape(1, -1)
image[x ** 2 + y ** 2 < 30 ** 2] = 0.5

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(
    image,
    origin="lower",
    cmap="plasma",
    extent=(-180, 180, -90, 90),
    aspect="auto",
)
plt.xticks(np.arange(-180, 180.1, 30))
plt.xlabel("longitude [deg]")
plt.yticks(np.arange(-90, 90.1, 30))
plt.ylabel("longitude [deg]")
plt.grid()
plt.colorbar();

Note that we scaled things so the photosphere has unit intensity and the spot has an intensity of `0.5`. Next, let's define our two spectra: one for the photosphere and one for the spot. For simplicity, we'll give each spectrum a single Gaussian absorption line, although the location will be different for each component.

In [ ]:
wav0 = np.linspace(642.0, 644.0, 400)
wav = np.linspace(642.5, 643.5, 200)
spec1 = 1 - np.exp(-0.5 * (wav0 - 642.8) ** 2 / 0.05 ** 2)
spec2 = 1 - np.exp(-0.5 * (wav0 - 643.2) ** 2 / 0.05 ** 2)

In [ ]:
plt.axvspan(wav[0], wav[-1], alpha=0.1, color="k")
plt.plot(wav0, spec1, label="photosphere")
plt.plot(wav0, spec2, label="spot")
plt.xlabel("rest frame wavelength [nm]")
plt.ylabel("normalized intensity")
plt.legend();

We're now ready to define the full spectral-spatial stellar surface. Let's create a data `cube`: a latitude-longitude grid with a third dimension that contains the spectrum at each point on the surface. We'll assign each pixel with unit intensity the photospheric spectrum (`spec1`) and the remaining pixels the spot spectrum (`spec2`):

In [ ]:
cube = np.zeros((len(lat), len(lon), len(wav0)))
bkg = image == 1
cube[bkg, :] = image[bkg].reshape(-1, 1) * spec1.reshape(1, -1)
cube[~bkg, :] = image[~bkg].reshape(-1, 1) * spec2.reshape(1, -1)

Here's what the (zoomed-in, low-res) `cube` looks like:

In [ ]:
# Downsample for plotting
from scipy.ndimage import zoom

mini_cube = zoom(cube[75:225, 200:400, :], [0.075, 0.075, 1])
nlat = mini_cube.shape[0]
nlon = mini_cube.shape[1]

cmap = plt.get_cmap("plasma")
fig, ax = plt.subplots(nlat, nlon, sharex=True, sharey=True, figsize=(12, 6))
fig.subplots_adjust(hspace=0, wspace=0)
for i in range(nlat):
    for j in range(nlon):
        ax[i, j].set_facecolor(cmap(mini_cube[i, j, 0]))
        ax[i, j].plot(mini_cube[i, j], lw=1, color="k")
        ax[i, j].set_xticks([])
        ax[i, j].set_yticks([])
        for s in ["top", "right", "bottom", "left"]:
            ax[i, j].spines[s].set_alpha(0.1)
        ax[i, j].margins(0.1, 0.1)
for i in range(nlat):
    ax[i, 0].spines["left"].set_alpha(1)
    ax[i, -1].spines["right"].set_alpha(1)
for j in range(nlon):
    ax[0, j].spines["top"].set_alpha(1)
    ax[-1, j].spines["bottom"].set_alpha(1)

## Instantiating the `starry` model

Now we're going to `load` this data cube into a `DopplerMap`. Let's instantiate our map as usual, specifying our rest frame and observed wavelength grids, `wav` and `wav0`. This time we'll provide a number of components for the map, `nc`. In this case, we only need two: one for the photosphere and one for the spot.

In [ ]:
map = starry.DopplerMap(15, nc=2, wav=wav, wav0=wav0)

For a little extra flavor, let's give the star a bit of inclination and a significant equatorial velocity of `30 km/s`.

In [ ]:
map.inc = 60
map.veq = 30000

Loading our data `cube` is extremely simple:

In [ ]:
map.load(cube=cube)

Internally, `starry` computes the singular value decomposition (SVD) of the cube to figure out the eigenmaps and eigenspectra defining the stellar surface. Let's take a look at what we have now:

In [ ]:
map.visualize()